In [ ]:
# GeoTrellis Operations

In [1]:
import astraea.spark.rasterframes._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

implicit val spark = SparkSession.builder().
  master("local[*]").appName("RasterFrames").getOrCreate().withRasterFrames
spark.sparkContext.setLogLevel("ERROR")
import spark.implicits._
val scene = SinglebandGeoTiff("../samples/L8-B8-Robinson-IL.tiff")
val rf = scene.projectedRaster.toRF(128, 128)

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.2:4041
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1530810905187)
SparkSession available as 'spark'


import astraea.spark.rasterframes._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@48f0a03c
import spark.implicits._
scene: geotrellis.raster.io.geotiff.SinglebandGeoTiff = SinglebandGeoTiff(geotrellis.raster.UShortConstantNoDataArrayTile@3b0587d8,Extent(431902.5, 4313647.5, 443512.5, 4321147.5),EPSG:32616,Tags(Map(AREA_OR_POINT -> POINT),List(Map())),GeoTiffOptions(geotrellis.raster.io.geotiff.Striped@4931bd7e,geotrellis.raster.io.geotiff.compression.DeflateCompression$@5e1dd8c7,1,None))
rf: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, t...

In [ ]:
GeoTrellis provides a [rich set of Map Algebra operations](https://docs.geotrellis.io/en/latest/guide/core-concepts.html#map-algebra) and other tile processing features that can be used with RasterFrames via Spark's UDF support.

Here's an example creating a UDFs to invoke the `equalize` transformation on each tile in a RasterFrame, and then compute the resulting per-tile mean of it.

In [2]:
import geotrellis.raster.equalization._
val equalizer = udf((t: Tile) => t.equalize())
val equalized = rf.select(equalizer($"tile") as "equalized")
equalized.select(tileMean($"equalized") as "equalizedMean").show(5, false)

+------------------+
|equalizedMean     |
+------------------+
|32634.257767197705|
|32777.45174876396 |
|32775.68302508698 |
|32881.48989806507 |
|32776.88976377953 |
+------------------+
only showing top 5 rows



import geotrellis.raster.equalization._
equalizer: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,org.apache.spark.sql.gt.types.TileUDT@67300a29,Some(List(org.apache.spark.sql.gt.types.TileUDT@67300a29)))
equalized: org.apache.spark.sql.DataFrame = [equalized: rf_tile]


In [ ]:
Here's an example downsampling a tile and rendering each tile as a matrix of numerical values.

In [3]:
val downsample = udf((t: Tile) => t.resample(4, 4))
val downsampled = rf.select(renderAscii(downsample($"tile")) as "minime")
downsampled.show(5, false)

spark.stop()

+-------------------+
|minime             |
+-------------------+
|∘∘∘∘
∘∘∘∘
∘∘∘∘
∘∘∘∘|
|.%*+
=@=@
#+:#
:-  |
|#:.%
* + 
=:-@
@#=+|
|-+%=
:*:.
=+ @
#@ #|
|-:*+
.=-.
 +@#
%.@#|
+-------------------+
only showing top 5 rows



downsample: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,org.apache.spark.sql.gt.types.TileUDT@67300a29,Some(List(org.apache.spark.sql.gt.types.TileUDT@67300a29)))
downsampled: org.apache.spark.sql.DataFrame = [minime: string]
